In [5]:
import pandas as pd
data = pd.read_csv('data_merged_updated_similarity_few_shot.csv', sep=';')


In [32]:
df = pd.read_csv('data_merged_updated_similarity_few_shot.csv', sep=';')


In [33]:
goqa = df[(df['source'] == 'GAS') | (df['source'] == 'WVS')]

In [18]:
data = data[(data['source'] == 'GPT-4 generated') |(data['source'] == 'GPT / https://www.beelovedcity.com/england-quiz')
            |(data['source'] == 'GPT / https://www.anadventurousworld.com/usa-trivia-questions/')]


In [19]:
import openai
import os
import pandas as pd
from openai import OpenAI

client = OpenAI(
    api_key=os.environ.get("OPENAI_API_KEY"),
)

def gpt4_text_similarity(text1, text2, model="gpt-4-1106-preview"):
    """
    Measures the similarity between two texts using GPT-4.

    Parameters:
    text1 (str): First text for comparison.
    text2 (str): Second text for comparison.
    model (str): The GPT model to use.

    Returns:
    float: A similarity score between 0 (not similar) and 1 (very similar).
    """

    system = f"On a scale between 1 and 5, how similar are the following two sentences? Respond only with a score between 1 and 5."
    prompt = f"""
                Examples:
                Input: 
                Sentence 1: Not really, too busy for that.
                Sentence 2: Yes, quite fond of academic journals.
                Output: 1.0
                Input: 
                Sentence 1: Universal healthcare, accessible to all residents.
                Sentence 2: NHS provides universal healthcare for all.
                Output: 2.0
                Input: 
                Sentence 1: Based on recommendations and personal interests.
                Sentence 2: By author, genre, recommendations, and reviews.
                Output: 3.0
                Input: 
                Sentence 1: As often as I can.
                Sentence 2: Quite often, I'm always willing.
                Output: 4.0
                Input: 
                Sentence 1: Yes, I love wearing hats!
                Sentence 2: Yes, I quite fancy wearing hats.
                Output: 5.0    
                \nInput:\n\nSentence 1: {text1}\n\n Sentence 2: {text2}. \n Output:"""
    #prompt = f"Rate the similarity between the following two texts on a scale from 0 (completely different) to 1 (identical):\n\nText 1: {text1}\n\nText 2: {text2}"
    response = client.chat.completions.create(
    model=model,
    messages=[
        {
        "role": "system",
        "content": system
        },
        {
        "role": "user",
        "content":  prompt
        }
    ],
    max_tokens=20
    )
    #print(response.choices[0].message.content)
    # Extracting the similarity score from the response
    try:
        last_message = response.choices[0].message.content
        similarity_score = float(last_message.strip())
    except (ValueError, KeyError, IndexError):
        similarity_score = None
    print(similarity_score)
    return similarity_score


In [13]:
def normalize_column(df, column_name, new_column_name):
    """
    Normalize the values in a DataFrame column to the range 0-1 and save them in a new column.
    
    Parameters:
    df (pandas.DataFrame): The DataFrame containing the column to normalize.
    column_name (str): The name of the column to normalize.
    new_column_name (str): The name of the new column for the normalized values.
    
    Returns:
    pandas.DataFrame: A DataFrame with the additional normalized column.
    """
    # Copy the DataFrame to avoid modifying the original data
    df_normalized = df.copy()
    
    # Apply Min-Max normalization
    df_normalized[new_column_name] = (df_normalized[column_name] - 1) / (5 - 1)
    
    return df_normalized

In [14]:
def add_small_value(df, column_name):
    """
    Add 0.00001 to each value in the specified column of a DataFrame 
    if the value is not 0.0 or 1.0.
    
    Parameters:
    df (pandas.DataFrame): The DataFrame containing the column.
    column_name (str): The name of the column to modify.
    
    Returns:
    pandas.DataFrame: A DataFrame with the modified column.
    """
    # Define the lambda function for the condition
    add_value = lambda x: x + 0.000001 if x not in [0.0, 1.0] else x

    # Apply the function to the specified column
    df[column_name] = df[column_name].apply(add_value)
    
    return df

In [28]:
data

,Unnamed: 0,question,selections,options,options_formatted,source,value_us,value_uk,answer_us,answer_uk,...,options_dict,score_ground_truth_answers,value_diff,question_type,ukGT_usGT_score,ukMA_usGT_score,ukGT_usMA_score,ukMA_usMA_score,us_score,uk_score
0,637,"In your view, how often do the following thing...","{'United States': [0.075, 0.272, 0.433, 0.1830...","['Very often', 'Fairly often', 'Not often', 'N...",(A) Very often\n(B) Fairly often\n(C) Not ofte...,WVS,"0,433001","0,386001",Not often,Not at all often,...,"{'A': 'Very often', 'B': 'Fairly often', 'C': ...",0.903704,953.0,Ordinal Scale,0.903705,1.000000,0.903705,1.000000,1.000000,0.903705
1,638,For each of the following statements I read ou...,"{'United States': [0.033, 0.174, 0.569, 0.217,...","['Agree strongly', 'Agree', 'Disagree', 'Stron...",(A) Agree strongly\n(B) Agree\n(C) Disagree\n(...,WVS,"0,569001","0,538001",Disagree,Disagree,...,"{'A': 'Agree strongly', 'B': 'Agree', 'C': 'Di...",969.000000,969.0,Ordinal Scale,0.969001,1.000000,1.000000,1.000000,1.000000,1.000000
2,639,On this list are various groups of people. Cou...,"{'United States': [0.127, 0.8109999999999999, ...","['Mentioned', 'Not mentioned', ""Don't know"", '...",(A) Mentioned\n(B) Not mentioned\n(C) Don't kn...,WVS,"0,811001","0,950001",Not mentioned,Not mentioned,...,"{'A': 'Mentioned', 'B': 'Not mentioned', 'C': ...",861.000000,861.0,Multiple Choice,0.861001,1.000000,1.000000,1.000000,1.000000,1.000000
3,640,I am going to name a number of organizations. ...,"{'United States': [0.111, 0.46, 0.350000000000...","['A great deal', 'Quite a lot', 'Not very much...",(A) A great deal\n(B) Quite a lot\n(C) Not ver...,WVS,"0,460001","0,492001",Quite a lot,Quite a lot,...,"{'A': 'A great deal', 'B': 'Quite a lot', 'C':...",968.000000,968.0,Ordinal Scale,0.968001,0.881482,1.000000,0.881482,1.000000,0.881482
4,641,How frequently do the following things occur i...,"{'United States': [0.055999999999999994, 0.108...","['Very Frequently', 'Quite frequently', 'Not f...",(A) Very Frequently\n(B) Quite frequently\n(C)...,WVS,"0,485001","0,430001",Not at all frequently,Not at all frequently,...,"{'A': 'Very Frequently', 'B': 'Quite frequentl...",945.000000,945.0,Ordinal Scale,0.945001,0.865673,0.865673,1.000000,0.865673,0.865673
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
986,1623,What was the first school founded in my country?,{},NaN,NaN,GPT / https://www.anadventurousworld.com/usa-t...,"0,000001","0,000001",Boston Latin School.,King's School Canterbury.,...,NaN,0.000000,1.0,Free-text,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
987,1624,When was the first public opening of a Disneyl...,{},NaN,NaN,GPT / https://www.anadventurousworld.com/usa-t...,"0,000001","0,000001","July 17, 1955.","July 17, 1955.",...,NaN,1.000000,1.0,Free-text,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
988,1625,Where was the first World Fair held in 1853?,{},NaN,NaN,GPT / https://www.anadventurousworld.com/usa-t...,"0,000001","0,000001",New York City.,"Crystal Palace, London.",...,NaN,0.000000,1.0,Free-text,0.000000,0.000000,0.000000,0.250001,0.000000,0.000000
989,1626,How many national championships has the most s...,{},NaN,NaN,GPT / https://www.anadventurousworld.com/usa-t...,"0,000001","0,000001",Too many to count.,Depends on the athlete.,...,NaN,0.000000,1.0,Numerical Scale,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000


In [21]:
print("uk_score Start")
def apply_similarity(row):
    try:
        return gpt4_text_similarity(row['model_answer_uk'], row['answer_uk'])
    except Exception as e:
        print(f"Error processing row: {e}")
        return None

data['uk_score'] = data.apply(apply_similarity, axis=1)
print("uk_score Finished")

print("us_score Start")
def apply_similarity(row):
    try:
        return gpt4_text_similarity(row['model_answer_us'], row['answer_us'])
    except Exception as e:
        print(f"Error processing row: {e}")
        return None

data['us_score'] = data.apply(apply_similarity, axis=1)

print("us_score finished")

print("ukGT_usGT_score start")
def apply_similarity(row):
    try:
        return gpt4_text_similarity(row['answer_uk'], row['answer_us'])
    except Exception as e:
        print(f"Error processing row: {e}")
        return None

data['ukGT_usGT_score'] = data.apply(apply_similarity, axis=1)

print("ukGT_usGT_score finished")

print("ukGT_usMA_score start")
def apply_similarity(row):
    try:
        return gpt4_text_similarity(row['answer_uk'], row['model_answer_us'])
    except Exception as e:
        print(f"Error processing row: {e}")
        return None

data['ukGT_usMA_score'] = data.apply(apply_similarity, axis=1)
print("ukGT_usMA_score finished")

print("ukMA_usGT_score start")

def apply_similarity(row):
    try:
        return gpt4_text_similarity(row['answer_us'], row['model_answer_uk'])
    except Exception as e:
        print(f"Error processing row: {e}")
        return None

data['ukMA_usGT_score'] = data.apply(apply_similarity, axis=1)
print("ukMA_usGT_score finished")

print("ukMA_usMA_score start")
def apply_similarity(row):
    try:
        return gpt4_text_similarity(row['model_answer_us'], row['model_answer_uk'])
    except Exception as e:
        print(f"Error processing row: {e}")
        return None

data['ukMA_usMA_score'] = data.apply(apply_similarity, axis=1)
print("ukMA_usMA_score finished")


uk_score Start
5.0
5.0
5.0
3.0
5.0
5.0
2.0
1.0
5.0
2.0
4.0
4.0
5.0
5.0
5.0
5.0
5.0
4.0
5.0
5.0
2.0
4.0
4.0
1.0
5.0
5.0
2.0
4.0
1.0
1.0
5.0
5.0
1.0
5.0
5.0
5.0
5.0
1.0
4.0
5.0
3.0
3.0
5.0
1.0
5.0
1.0
2.0
5.0
3.0
1.0
1.0
3.0
3.0
3.5
5.0
1.0
3.0
3.0
1.0
2.0
3.0
3.0
4.0
2.0
2.0
1.0
2.0
2.0
5.0
5.0
4.0
5.0
1.0
4.0
5.0
3.0
5.0
5.0
1.0
4.0
5.0
5.0
2.0
3.0
3.0
5.0
4.0
1.0
5.0
3.0
4.0
5.0
5.0
3.0
3.0
5.0
5.0
3.0
1.0
5.0
1.0
1.0
5.0
1.0
5.0
5.0
4.0
1.0
2.0
2.0
5.0
5.0
2.0
3.0
1.0
1.0
4.0
5.0
3.0
1.0
2.0
4.0
1.0
5.0
5.0
5.0
5.0
1.0
3.0
5.0
5.0
1.0
1.0
5.0
2.0
5.0
5.0
1.0
2.0
1.0
1.0
1.0
2.0
5.0
1.0
1.0
1.0
5.0
4.0
4.0
1.0
2.0
4.0
5.0
4.0
5.0
5.0
3.0
5.0
3.0
3.0
5.0
5.0
5.0
1.0
4.0
5.0
4.0
1.0
4.0
5.0
4.0
4.0
1.0
4.0
5.0
2.0
1.0
2.0
4.0
4.0
5.0
5.0
1.0
1.0
2.0
5.0
1.0
4.0
5.0
4.0
5.0
1.0
4.0
5.0
4.0
1.0
1.0
1.0
5.0
4.0
4.0
1.0
5.0
1.0
4.0
1.0
4.0
2.0
5.0
1.0
1.0
4.0
2.0
4.0
4.0
1.0
3.0
1.0
4.0
2.0
1.0
5.0
4.0
5.0
2.0
2.0
2.0
5.0
1.0
2.0
5.0
1.0
4.0
1.0
4.0
1.0
4.0
4.0
4.0
3.0
1.0
2.0
1.0
1.0
2.0
4

/var/folders/tq/vxytrqv13kx3fp71v2wgtd000000gn/T/ipykernel_87887/476571658.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['uk_score'] = data.apply(apply_similarity, axis=1)


1.0
1.0
5.0
4.0
5.0
5.0
4.0
5.0
5.0
4.0
4.0
5.0
2.0
4.0
5.0
5.0
5.0
5.0
3.0
5.0
5.0
4.0
5.0
5.0
3.0
5.0
2.0
5.0
5.0
2.0
5.0
2.0
5.0
2.0
5.0
5.0
1.0
5.0
4.0
5.0
1.0
5.0
5.0
4.0
5.0
1.0
5.0
5.0
4.0
1.0
5.0
1.0
2.0
4.0
5.0
5.0
2.0
2.0
3.0
4.0
3.0
2.0
5.0
4.0
2.0
5.0
2.0
2.0
1.0
1.0
1.0
5.0
1.0
2.0
4.0
3.0
5.0
5.0
3.0
1.0
5.0
1.0
3.0
2.0
1.0
4.0
4.0
2.0
5.0
2.0
4.0
5.0
5.0
3.0
5.0
5.0
1.0
2.0
2.0
4.0
1.0
1.0
1.0
1.0
5.0
2.0
5.0
2.0
1.0
5.0
5.0
5.0
1.0
2.0
2.0
1.0
3.0
1.0
3.0
3.0
3.0
5.0
5.0
2.0
3.0
5.0
5.0
5.0
5.0
1.0
5.0
3.0
1.0
4.0
4.0
1.0
5.0
5.0
2.0
3.0
1.0
4.0
5.0
4.0
1.0
2.0
1.0
5.0
5.0
5.0
5.0
2.0
2.0
1.0
4.0
5.0
1.0
1.0
5.0
4.0
1.0
5.0
4.0
4.0
5.0
5.0
4.0
3.0
1.0
5.0
4.0
3.0
5.0
4.0
5.0
4.0
3.0
1.0
5.0
1.0
4.0
5.0
1.0
4.0
2.0
1.0
5.0
5.0
3.0
1.0
1.0
5.0
3.0
4.0
5.0
4.0
2.0
5.0
1.0
1.0
2.0
5.0
4.0
4.0
5.0
5.0
4.0
5.0
5.0
5.0
5.0
1.0
4.0
1.0
1.0
4.0
1.0
1.0
1.0
1.0
1.0
1.0
5.0
2.0
5.0
1.0
4.0
4.0
5.0
1.0
2.0
5.0
5.0
4.0
5.0
1.0
5.0
4.0
1.0
1.0
5.0
2.0
5.0
1.0
5.0
2.0
5.0
5.0
4.0
1.0


/var/folders/tq/vxytrqv13kx3fp71v2wgtd000000gn/T/ipykernel_87887/476571658.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['us_score'] = data.apply(apply_similarity, axis=1)


4.0
2.0
5.0
4.0
4.0
5.0
3.0
2.0
1.0
3.0
2.0
4.0
5.0
5.0
4.0
2.0
1.0
5.0
3.0
4.0
2.0
4.0
5.0
1.0
1.0
1.0
2.0
1.0
3.0
1.0
4.0
2.0
1.0
2.0
5.0
4.0
2.0
1.0
2.0
5.0
5.0
4.0
1.0
1.0
1.0
1.0
2.0
1.0
3.0
3.0
1.0
2.0
3.0
2.0
4.0
5.0
1.0
2.0
2.0
4.0
2.0
3.0
1.0
1.0
1.0
1.0
1.0
1.0
4.0
1.0
3.0
1.0
1.0
1.0
4.0
2.0
2.0
1.0
2.0
1.0
4.0
1.0
3.0
2.0
1.0
4.0
3.0
1.0
4.0
1.0
2.0
1.0
1.0
2.0
1.0
1.0
5.0
1.0
1.0
1.0
1.0
1.0
5.0
1.0
5.0
2.0
4.0
1.0
2.0
4.0
1.0
1.0
1.0
2.0
1.0
4.0
1.0
1.0
2.0
1.0
2.0
3.0
4.0
1.0
1.0
5.0
5.0
2.0
2.0
3.0
5.0
4.0
1.0
5.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
2.0
4.0
1.0
5.0
2.0
4.0
1.0
4.0
2.0
1.0
4.0
1.0
5.0
5.0
1.0
1.0
2.0
4.0
3.0
2.0
4.0
5.0
2.0
5.0
4.0
4.0
4.0
4.0
4.0
4.0
1.0
4.0
2.0
1.0
5.0
1.0
5.0
2.0
1.0
3.0
5.0
4.0
3.0
5.0
1.0
4.0
5.0
4.0
1.0
1.0
5.0
3.0
2.0
5.0
2.0
1.0
2.0
2.0
1.0
1.0
1.0
3.0
4.0
5.0
4.0
1.0
4.0
1.0
3.0
1.0
3.0
4.0
1.0
1.0
4.0
5.0
1.0
4.0
1.0
1.0
1.0
5.0
3.0
1.0
2.0
2.0
1.0
5.0
5.0
4.0
5.0
1.0
4.0
1.0
1.0
1.0
4.0
2.0
1.0
4.0
3.0
5.0
3.0
4.0
3.0
2.0
5.0
4.0
1.0


...

/var/folders/tq/vxytrqv13kx3fp71v2wgtd000000gn/T/ipykernel_87887/476571658.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['ukGT_usMA_score'] = data.apply(apply_similarity, axis=1)


4.0
2.0
5.0
2.0
4.0
5.0
3.0
5.0
1.0
3.0
2.0
4.0
5.0
5.0
5.0
2.0
2.0
2.0
2.0
5.0
1.0
5.0
3.0
1.0
1.0
1.0
4.0
1.0
1.0
1.0
4.0
2.0
5.0
2.0
5.0
3.0
2.0
1.0
1.0
5.0
4.0
4.0
1.0
1.0
1.0
1.0
1.0
1.0
2.0
4.0
1.0
3.0
3.0
4.0
5.0
1.0
3.0
3.0
1.0
2.0
3.0
2.0
1.0
2.0
2.0
2.0
1.0
1.0
4.0
1.0
2.0
1.0
1.0
1.0
4.0
3.0
1.0
1.0
1.0
1.0
5.0
1.0
3.0
2.0
1.0
4.0
2.0
1.0
4.0
1.0
1.0
1.0
1.0
2.0
1.0
1.0
5.0
1.0
1.0
1.0
1.0
5.0
5.0
1.0
5.0
2.0
4.0
2.0
1.0
2.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
2.0
2.0
1.0
2.0
1.0
4.0
5.0
1.0
5.0
3.0
4.0
1.0
1.0
5.0
1.0
1.0
2.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
4.0
5.0
4.0
3.0
1.0
5.0
3.0
1.0
1.0
1.0
4.0
5.0
2.0
5.0
4.0
1.0
1.0
3.0
4.0
2.0
1.0
5.0
4.0
1.0
3.0
1.0
1.0
4.0
2.0
1.0
5.0
1.0
2.0
5.0
5.0
1.0
1.0
1.0
5.0
1.0
3.0
1.0
1.0
5.0
1.0
4.0
5.0
4.0
2.0
1.0
2.0
1.0
1.0
3.0
1.0
4.0
1.0
4.0
1.0
5.0
5.0
2.0
5.0
1.0
4.0
1.0
1.0
4.0
1.0
1.0
1.0
1.0
1.0
1.0
5.0
4.0
1.0
1.0
1.0
1.0
5.0
1.0
2.0
5.0
5.0
4.0
5.0
1.0
5.0
3.0
1.0
1.0
2.0
1.0
4.0
1.0
1.0
4.0
5.0
5.0
1.0
2.0


/var/folders/tq/vxytrqv13kx3fp71v2wgtd000000gn/T/ipykernel_87887/476571658.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['ukMA_usGT_score'] = data.apply(apply_similarity, axis=1)


1.0
2.0
5.0
4.0
4.0
5.0
4.0
5.0
1.0
4.0
2.0
4.0
1.0
4.0
5.0
2.0
1.0
3.0
3.0
5.0
2.0
4.0
5.0
1.0
1.0
1.0
3.0
1.0
1.0
1.0
4.0
1.0
5.0
4.0
5.0
3.0
1.0
1.0
1.0
5.0
1.0
3.0
1.0
1.0
1.0
1.0
1.0
1.0
2.0
1.0
1.0
2.0
5.0
4.0
5.0
1.0
2.0
3.0
1.0
2.0
3.0
2.0
1.0
4.0
4.0
2.0
2.0
1.0
1.0
1.0
1.0
1.0
5.0
1.0
4.0
5.0
1.0
1.0
1.0
5.0
4.0
1.0
2.0
3.0
1.0
5.0
3.0
1.0
4.0
2.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
2.0
1.0
1.0
1.0
1.0
1.0
5.0
5.0
5.0
2.0
1.0
2.0
1.0
1.0
1.0
1.0
1.0
4.0
1.0
1.0
1.0
1.0
2.0
2.0
1.0
5.0
1.0
5.0
5.0
1.0
5.0
4.0
5.0
1.0
5.0
4.0
1.0
4.0
2.0
2.0
1.0
1.0
2.0
1.0
2.0
1.0
4.0
1.0
5.0
1.0
4.0
5.0
4.0
3.0
2.0
1.0
5.0
1.0
4.0
2.0
5.0
4.0
5.0
5.0
5.0
1.0
1.0
5.0
4.0
5.0
5.0
5.0
5.0
1.0
4.0
1.0
1.0
4.0
2.0
5.0
5.0
5.0
4.0
5.0
1.0
2.0
2.0
2.0
4.0
1.0
4.0
5.0
4.0
5.0
1.0
5.0
5.0
5.0
4.0
1.0
1.0
3.0
1.0
2.0
2.0
5.0
1.0
4.0
1.0
5.0
5.0
2.0
5.0
5.0
5.0
5.0
2.0
5.0
5.0
3.0
4.0
5.0
5.0
4.0
5.0
5.0
1.0
5.0
2.0
1.0
5.0
4.0
2.0
5.0
5.0
4.0
5.0
5.0
5.0
5.0
1.0
1.0
2.0
1.0
4.0
5.0
1.0
2.0
5.0
5.0
4.0
5.0


/var/folders/tq/vxytrqv13kx3fp71v2wgtd000000gn/T/ipykernel_87887/476571658.py:67: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['ukMA_usMA_score'] = data.apply(apply_similarity, axis=1)


In [23]:
data.columns

Index(['Unnamed: 0', 'question', 'selections', 'options', 'options_formatted',
       'source', 'value_us', 'value_uk', 'answer_us', 'answer_uk', 'category',
       '# of options', 'question type', 'category_group', 'model_answer_us',
       'model_answer_uk', 'model_answer_uk_option_match',
       'model_answer_us_option_match', '#_options', 'options_dict',
       'score_ground_truth_answers', 'value_diff', 'question_type',
       'ukGT_usGT_score', 'ukMA_usGT_score', 'ukGT_usMA_score',
       'ukMA_usMA_score', 'us_score', 'uk_score'],
      dtype='object')

In [24]:
data = normalize_column(data, 'us_score', 'us_score')
data = normalize_column(data, 'uk_score', 'uk_score')
data = normalize_column(data, 'ukGT_usGT_score', 'ukGT_usGT_score')
data = normalize_column(data, 'ukMA_usGT_score', 'ukMA_usGT_score')
data = normalize_column(data, 'ukGT_usMA_score', 'ukGT_usMA_score')
data = normalize_column(data, 'ukMA_usMA_score', 'ukMA_usMA_score')
data = add_small_value(data, 'us_score')
data = add_small_value(data, 'uk_score')
data = add_small_value(data, 'ukGT_usGT_score')
data = add_small_value(data, 'ukMA_usGT_score')
data = add_small_value(data, 'ukGT_usMA_score')
data = add_small_value(data, 'ukMA_usMA_score')


In [35]:
merged_df = pd.concat([goqa, data], ignore_index=True)


In [37]:
inspect = merged_df[merged_df['ukGT_usGT_score'] < 0.9]
print("-----------------")
print("US-Score:", inspect.us_score.mean())
print("UK-Score:", inspect.uk_score.mean())
print("GroundTruth-Score:", inspect.ukGT_usGT_score.mean())
print("UK-ModelAnswer_vs_US-GroundTruth-Score:", inspect.ukMA_usGT_score.mean())
print("US-ModelAnswer_vs_UK-GroundTruth-Score:", inspect.ukGT_usMA_score.mean())
print("UK-ModelAnswer_vs_US-ModelAnswer-Score:", inspect.ukMA_usMA_score.mean())
print("Data size:", len(inspect))
print("-----------------")

-----------------
US-Score: 0.5751162591209428
UK-Score: 0.5612361412937029
GroundTruth-Score: 0.4301639327324609
UK-ModelAnswer_vs_US-GroundTruth-Score: 0.46900712058759736
US-ModelAnswer_vs_UK-GroundTruth-Score: 0.4636023074677981
UK-ModelAnswer_vs_US-ModelAnswer-Score: 0.7145111496862704
Data size: 1015
-----------------
